In [3]:
import os
path_test_cases = '../test_cases_for_task1'

documents = []

for filename in os.listdir(path_test_cases):
    file_path = os.path.join(path_test_cases, filename)
    if os.path.isfile(file_path): # Double ckeck that file path exist and valid
        with open(file_path, 'r') as file:
            content = file.read()
            documents.append(content)

In [17]:
import spacy
from spacy.lang.ru.stop_words import STOP_WORDS

nlp = spacy.load('ru_core_news_sm')

for text in documents:
    tmp = nlp(text.lower()) 
    
    filtered_words = []
    for word in tmp:
        if not word.is_stop and not word.is_punct:  
            filtered_words.append(word.lemma_)
    
    print(filtered_words) 
    
    break

['неделя', 'обсуждать', 'возможность', 'покупка', 'оборудование', 'конкретный', 'решение', 'принять', 'следующий', 'совещание', 'назначить', 'следующий', 'понедельник']
